## 1. IMPORTING PYTHON LIBRARIES

In [52]:
# to work with data: dataframes, statistics & regular expressions
import pandas as pd
import numpy as np
import re
import pandas_profiling as pdp # suggested by Dani in Slack
from geopy import distance # to recalculate distance

## 2. DATA WRANGLING

In [2]:
# import CSV :
def importing_csv(csv_path):
    df = pd.read_csv(csv_path)
    return df

# to get a overview & description of the dataset: shape, columns name, info about variables type and registers or statistics per variable :
def raw(df):
    print('shape:',df.shape)
    print('\n columns:',df.columns)
    print('\n variables info:')    
    return df.info(),df.describe()

# to replace NaN with 0 in numeric variables :
def replace_Nan_with0(df,num_var):
    for nv in num_var: 
        df[nv].fillna("0", inplace = True, downcast='infer')
    return df[num_var]

# to convert to category type :
def convert_to_category(df,cat_var):
    for cv in cat_var: 
        df[cv] = df[cv].astype('category')
    return df.dtypes

# to convert a single column to int type :
def convert_col_to_int(col):
    col = col.apply(lambda x: x.astype('int64', copy=False))
    return type(col)

# to convert multiple columns to int type :
def convert_to_int(df,int_var):
    for iv in int_var:
        df[iv] = df[iv].apply(lambda x: x.astype('int64', copy=False))
    return df.dtypes

# to convert a single column to numeric (int or float type - if some NaN)
def convert_col_to_numeric(col):
    col = col.apply(pd.to_numeric, errors='coerce')
    return type(col)

# to convert multiple columns to numeric (int or float type - if some NaN)
def convert_col_to_numeric(df,num_var):
    for nv in num_var:
        df[nv] = df[nv].apply(pd.to_numeric, errors='coerce')
    return df.dtypes

# other way to convert a single column to float type (only if no NaN)
def convert_col_to_float2(col):
    col = col.apply.astype('float64', copy=False)
    return type(col)

# other way to convert multiple columns to float type (only if no NaN)
def convert_to_float2(df,float_var):
    for fv in float_var:
        df[fv] = df[fv].apply(lambda x: x.astype('str').astype('float64', copy=False))
    return df.dtypes

def split_datetime(df,datetime):
    year,month,date,time = str(datetime+'_year'),str(datetime+'_month'),str(datetime+'_date'),str(datetime+'_time')
    df[year] = pd.to_datetime(df[datetime]).dt.year
    df[month] = pd.to_datetime(df[datetime]).dt.month
    df[date] = pd.to_datetime(df[datetime]).dt.date
    df[time] = pd.to_datetime(df[datetime]).dt.time
    return df[[datetime,year,month,date,time]]

def second_to_min_value(x):
    return x/60

def second_to_min(df,seconds_var):
    for var_sec in seconds_var:
        var_min = str(var_sec+'_min')
        df[var_min]=df[var_sec].apply(second_to_min_value)
    return df.describe()

# to remover comma from string:
def remove_comma(x):
     return str(x).replace(',','').strip()

# to replace comma per dot:
def replace_comma_per_dot(x):
    return str(x).replace(',','.').strip()

# to convert cents MXN to MXN:
def cents_to_MXN(x):
    return x/100

# to describe statistics measures for multiple (numeric) variables:
def stats_num(df,num_var):
    return df[num_var].describe()

# to counts values for multiple variables:
def valcount(df, var):
        return [df[v].value_counts() for v in var]

# to get top N for multiple variables:
def top_N(df, var, n):
        return [df[v].value_counts().head(n) for v in var]
    
# to export clean_csv
def export_clean_csv(df,csv_name):
    return (df).to_csv(str(csv_name+'.csv'))

In [3]:
# importing csv
csv_path = './db/tableau_project.csv'
data = importing_csv(csv_path)

In [4]:
# using my function to explore variables as overview: 
# shape, columns name, variables info (# registers per variable & type of variable) and statistics desciption for numeric variables.
raw(data)

shape: (3864, 22)

 columns: Index(['Unnamed: 0', 'journey_id', 'vehicle_type_id', 'start_type', 'start_at',
       'start_lat', 'start_lon', 'end_at', 'end_lat', 'end_lon', 'end_state',
       'price', 'price_distance', 'price_duration', 'distance', 'duration',
       'source', 'rider_waiting_time', 'driver_waiting_time',
       'price_supplements', 'discount', 'rating'],
      dtype='object')

 variables info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3864 entries, 0 to 3863
Data columns (total 22 columns):
Unnamed: 0             3864 non-null int64
journey_id             3864 non-null object
vehicle_type_id        3864 non-null object
start_type             3864 non-null object
start_at               3864 non-null object
start_lat              3864 non-null float64
start_lon              3864 non-null float64
end_at                 3864 non-null object
end_lat                3807 non-null float64
end_lon                3807 non-null float64
end_state              3864 non-nu

(None,
         Unnamed: 0    start_lat    start_lon      end_lat      end_lon  \
 count  3864.000000  3864.000000  3864.000000  3807.000000  3807.000000   
 mean   1931.500000    19.412159   -99.195443    19.410835   -99.197221   
 std    1115.585048     0.028146     0.040832     0.030001     0.044896   
 min       0.000000    19.290513   -99.288274    19.196463   -99.655665   
 25%     965.750000    19.405320   -99.207814    19.379486   -99.253980   
 50%    1931.500000    19.422749   -99.175834    19.422749   -99.178435   
 75%    2897.250000    19.426613   -99.170483    19.429759   -99.171955   
 max    3863.000000    19.513544   -99.051114    19.517451   -99.048817   
 
             rating  
 count  3864.000000  
 mean      8.289596  
 std       1.265744  
 min       4.000000  
 25%       8.000000  
 50%       9.000000  
 75%       9.000000  
 max      10.000000  )

In [5]:
# checking # NaN per variable
data.isna().sum()

Unnamed: 0                0
journey_id                0
vehicle_type_id           0
start_type                0
start_at                  0
start_lat                 0
start_lon                 0
end_at                    0
end_lat                  57
end_lon                  57
end_state                 0
price                    20
price_distance         1052
price_duration         1052
distance                 20
duration                 20
source                    0
rider_waiting_time     1131
driver_waiting_time    1179
price_supplements         0
discount                 20
rating                    0
dtype: int64

In [6]:
# checking how structured is the dataset looking for unique mentions per variable
data.nunique()

Unnamed: 0             3864
journey_id             3864
vehicle_type_id           2
start_type                2
start_at               3861
start_lat              1403
start_lon              1356
end_at                 3864
end_lat                 498
end_lon                 493
end_state                 5
price                  1417
price_distance         2170
price_duration          585
distance               2234
duration               1606
source                    3
rider_waiting_time     1076
driver_waiting_time     628
price_supplements        32
discount                  3
rating                    6
dtype: int64

In [7]:
# just for testing this library suggested by Dani in Slack **very useful, by the way**
pdp.ProfileReport(data)

Number of variables,22
Number of observations,3864
Total Missing (%),5.4%
Total size in memory,664.2 KiB
Average record size in memory,176.0 B
Numeric,6
Categorical,14
Boolean,0
Date,0
Text (Unique),2
Rejected,0


In [8]:
# first look at data inside variables:
data.head()

,Unnamed: 0,journey_id,vehicle_type_id,start_type,start_at,start_lat,start_lon,end_at,end_lat,end_lon,...,price_distance,price_duration,distance,duration,source,rider_waiting_time,driver_waiting_time,price_supplements,discount,rating
0,0,93e6d216088af74c32183283d4bb2953,077866c3fd1a75f51ca7f8eae166ae32,asap,2017-12-16 11:08:59,19.431363,-99.195362,2017-12-16 11:37:32,19.406438,-99.175260,...,"5,204",0,"7,943","1,058",Android,590,48,662,0,9.0
1,1,444b084ec64244919afadfba879876a3,077866c3fd1a75f51ca7f8eae166ae32,asap,2017-12-16 11:06:16,19.431567,-99.195368,2017-12-16 11:07:21,NaN,NaN,...,0,0,0,0,iPhone,NaN,NaN,0,0,9.0
2,2,dbcc57f4e7606a71c74c6e4666a977a4,077866c3fd1a75f51ca7f8eae166ae32,asap,2017-11-24 22:58:37,19.422052,-99.174499,2017-11-24 23:27:05,19.438666,-99.180821,...,"3,552",0,"3,054",689,Android,923,4,0,0,10.0
3,3,1ffe9a24033847148c683574985df56c,077866c3fd1a75f51ca7f8eae166ae32,asap,2017-12-16 09:17:08,19.433119,-99.154844,2017-12-16 09:17:30,19.290148,-99.144407,...,0,0,0,0,iPhone,NaN,NaN,0,0,9.0
4,4,fb1ef7d070724482a33be41f27737ea5,077866c3fd1a75f51ca7f8eae166ae32,asap,2017-11-24 18:11:00,19.440996,-99.183581,2017-11-24 18:31:31,19.422815,-99.174865,...,"2,974",12,"2,859",887,iPhone,329,36,0,0,8.0


In [9]:
# as we can not see all columns, we will set display option for columns:
pd.set_option('display.max_columns', 50)
data.head()

,Unnamed: 0,journey_id,vehicle_type_id,start_type,start_at,start_lat,start_lon,end_at,end_lat,end_lon,end_state,price,price_distance,price_duration,distance,duration,source,rider_waiting_time,driver_waiting_time,price_supplements,discount,rating
0,0,93e6d216088af74c32183283d4bb2953,077866c3fd1a75f51ca7f8eae166ae32,asap,2017-12-16 11:08:59,19.431363,-99.195362,2017-12-16 11:37:32,19.406438,-99.175260,drop off,"5,866","5,204",0,"7,943","1,058",Android,590,48,662,0,9.0
1,1,444b084ec64244919afadfba879876a3,077866c3fd1a75f51ca7f8eae166ae32,asap,2017-12-16 11:06:16,19.431567,-99.195368,2017-12-16 11:07:21,NaN,NaN,rider cancel,0,0,0,0,0,iPhone,NaN,NaN,0,0,9.0
2,2,dbcc57f4e7606a71c74c6e4666a977a4,077866c3fd1a75f51ca7f8eae166ae32,asap,2017-11-24 22:58:37,19.422052,-99.174499,2017-11-24 23:27:05,19.438666,-99.180821,drop off,4,"3,552",0,"3,054",689,Android,923,4,0,0,10.0
3,3,1ffe9a24033847148c683574985df56c,077866c3fd1a75f51ca7f8eae166ae32,asap,2017-12-16 09:17:08,19.433119,-99.154844,2017-12-16 09:17:30,19.290148,-99.144407,rider cancel,0,0,0,0,0,iPhone,NaN,NaN,0,0,9.0
4,4,fb1ef7d070724482a33be41f27737ea5,077866c3fd1a75f51ca7f8eae166ae32,asap,2017-11-24 18:11:00,19.440996,-99.183581,2017-11-24 18:31:31,19.422815,-99.174865,drop off,4,"2,974",12,"2,859",887,iPhone,329,36,0,0,8.0


In [10]:
# checking discount column value counts:
data['discount'].value_counts()

0        3842
6,317       1
4           1
Name: discount, dtype: int64

In [11]:
# dropping non relevant columns from dataset
data = data.drop(columns=["Unnamed: 0","discount"])

In [12]:
# replace Nan with '0' value:
#num_var = ['distance','driver_waiting_time','rider_waiting_time','price','price_duration','duration','price_distance','price_supplements']
#replace_Nan_with0(data,num_var)

In [13]:
# checking again NaN --> OK 
data.isna().sum()

journey_id                0
vehicle_type_id           0
start_type                0
start_at                  0
start_lat                 0
start_lon                 0
end_at                    0
end_lat                  57
end_lon                  57
end_state                 0
price                    20
price_distance         1052
price_duration         1052
distance                 20
duration                 20
source                    0
rider_waiting_time     1131
driver_waiting_time    1179
price_supplements         0
rating                    0
dtype: int64

In [14]:
# first we convert categorical variable to category format: 
cat_var = ['vehicle_type_id', 'start_type', 'source','end_state']

In [15]:
valcount(data, cat_var)

[21620ea5749f2e0679a8c72c7fbafb9e    3642
 077866c3fd1a75f51ca7f8eae166ae32     222
 Name: vehicle_type_id, dtype: int64, asap        3615
 reserved     249
 Name: start_type, dtype: int64, iPhone     2631
 Android    1218
 web          15
 Name: source, dtype: int64, drop off         2685
 rider cancel     1146
 not found          20
 no show            12
 driver cancel       1
 Name: end_state, dtype: int64]

In [16]:
data.loc[data.vehicle_type_id == '21620ea5749f2e0679a8c72c7fbafb9e', 'vehicle_type_id'] = 'A'
data.loc[data.vehicle_type_id == '077866c3fd1a75f51ca7f8eae166ae32', 'vehicle_type_id'] = 'B'

In [17]:
valcount(data, cat_var)

[A    3642
 B     222
 Name: vehicle_type_id, dtype: int64, asap        3615
 reserved     249
 Name: start_type, dtype: int64, iPhone     2631
 Android    1218
 web          15
 Name: source, dtype: int64, drop off         2685
 rider cancel     1146
 not found          20
 no show            12
 driver cancel       1
 Name: end_state, dtype: int64]

In [18]:
convert_to_category(data,cat_var)

journey_id               object
vehicle_type_id        category
start_type             category
start_at                 object
start_lat               float64
start_lon               float64
end_at                   object
end_lat                 float64
end_lon                 float64
end_state              category
price                    object
price_distance           object
price_duration           object
distance                 object
duration                 object
source                 category
rider_waiting_time       object
driver_waiting_time      object
price_supplements        object
rating                  float64
dtype: object

In [19]:
price_context = ['end_state','price','price_distance','price_duration','distance','duration','rider_waiting_time','driver_waiting_time', 'price_supplements']

In [20]:
data.loc[data['end_state'] == 'no show',price_context]
# here we see that 'no show' is when the rider had been waiting and there is a total price mainly based on duration
# hipotesis 1 : the ride took place (distance and duration) but the machine did not register the end_state.
# hipotesis 2 : the ride did not took place because rider did not show but the ride was paid due to cancelation policy (special offer that cannot be cancelled).

,end_state,price,price_distance,price_duration,distance,duration,rider_waiting_time,driver_waiting_time,price_supplements
80,no show,4,0,283,12,368,344,NaN,0
91,no show,4,0,"1,306",0,612,"9,673",NaN,0
302,no show,4,0,"2,354",29,865,354,NaN,0
1259,no show,"8,484",0,"8,484",53,"2,329",173,NaN,0
1276,no show,4,0,0,38,123,756,NaN,0
1460,no show,"6,93",0,"6,93",38,"1,96",357,NaN,0
1765,no show,0,0,"1,365","1,855",624,43,NaN,0
2012,no show,4,0,"1,268","1,594",602,168,NaN,0
2215,no show,"10,177",0,"10,177",5,"2,737",529,NaN,0
3421,no show,"4,5",0,0,6,246,334,NaN,0


In [21]:
data.loc[data['end_state'] == 'not found',price_context]
# here we see that 'not found' is when the driver did not find the rider so no ride took place.

,end_state,price,price_distance,price_duration,distance,duration,rider_waiting_time,driver_waiting_time,price_supplements
177,not found,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
602,not found,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
611,not found,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
612,not found,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
632,not found,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
752,not found,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
765,not found,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
814,not found,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
857,not found,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1106,not found,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [22]:
data.loc[data['price'] == '4',price_context].head(20)

,end_state,price,price_distance,price_duration,distance,duration,rider_waiting_time,driver_waiting_time,price_supplements
2,drop off,4,"3,552",0,"3,054",689,923,4,0
4,drop off,4,"2,974",12,"2,859",887,329,36,0
5,drop off,4,"3,346",290,"3,176","1,028",73,369,0
12,drop off,4,"3,766",0,"2,766",841,633,97,0
18,drop off,4,"2,912",0,"3,187",930,576,180,0
21,drop off,4,"3,179",0,"3,073","1,259",873,10,0
22,drop off,4,"3,108",0,"2,923","1,485",205,262,0
23,drop off,4,"3,643",0,"2,516",783,856,7,0
26,drop off,4,"3,644",0,"2,714",838,163,6,0
32,drop off,4,836,0,701,746,"1,171",160,0


In [23]:
data.loc[data['price'] == '4,5',price_context].head(20)

,end_state,price,price_distance,price_duration,distance,duration,rider_waiting_time,driver_waiting_time,price_supplements
1987,drop off,"4,5","3,872",0,"3,427",872,431,33,0
1988,drop off,"4,5","4,452",0,"1,275",392,502,3,0
1993,rider cancel,"4,5",0,0,0,0,937,NaN,0
3260,drop off,"4,5","2,338",0,"1,716",410,380,211,0
3261,drop off,"4,5","1,957","1,669","1,418",956,206,700,0
3268,drop off,"4,5","2,581",0,"4,203",755,430,214,0
3276,drop off,"4,5","1,784",0,"1,797",684,426,48,0
3284,drop off,"4,5","4,147",0,"3,173",557,447,13,0
3285,drop off,"4,5","1,918",0,"1,438",274,843,26,0
3288,drop off,"4,5","4,479",0,"3,295",614,275,2,0


In [24]:
data.loc[data['price'] == '10',price_context]

,end_state,price,price_distance,price_duration,distance,duration,rider_waiting_time,driver_waiting_time,price_supplements
66,rider cancel,10,0,0,0,0,234,NaN,0
70,drop off,10,"3,184","1,387","2,742","1,582","2,709",632,0
125,drop off,10,"3,16",0,"2,656",862,"2,708",193,0
135,rider cancel,10,0,0,0,0,"1,587",NaN,0
187,rider cancel,10,NaN,NaN,0,0,NaN,NaN,0
245,drop off,10,"3,305","1,367","2,914","1,323","3,273",627,0
286,rider cancel,10,NaN,NaN,0,0,NaN,NaN,0
299,rider cancel,10,NaN,NaN,0,0,NaN,NaN,0
387,drop off,10,0,234,10,394,"2,838",356,0
464,drop off,10,"3,212",0,"2,312",671,"1,794",203,0


In [25]:
data.loc[data['price_duration'] == '5',price_context]

,end_state,price,price_distance,price_duration,distance,duration,rider_waiting_time,driver_waiting_time,price_supplements
799,drop off,4,761,5,622,436,142,301,0
1085,drop off,"10,151","6,146",5,"6,205","2,737","2,619",424,0
1773,drop off,4,"3,592",5,"2,962",561,257,27,0
2067,drop off,4,"3,853",5,"2,975",918,21,301,0
2806,drop off,"9,049","9,044",5,"7,789","1,978",355,2,0


In [26]:
top_N(data, price_context, 10)

[drop off         2685
 rider cancel     1146
 not found          20
 no show            12
 driver cancel       1
 Name: end_state, dtype: int64, 0        1083
 4        1064
 4,5       103
 10         14
 4,404       5
 4,225       5
 4,547       4
 4,573       4
 4,217       4
 4,424       4
 Name: price, dtype: int64, 0         141
 3,453      14
 3,259       6
 4,404       5
 4,225       5
 3,335       5
 3,361       5
 4,547       4
 14,136      4
 4,429       4
 Name: price_distance, dtype: int64, 0      2087
 5         5
 24        5
 20        4
 142       4
 79        4
 120       4
 4         3
 32        3
 115       3
 Name: price_duration, dtype: int64, 0        1141
 3           5
 3,241       5
 3,031       4
 3,268       4
 3,392       4
 2,802       4
 3,198       4
 3,295       4
 3,147       4
 Name: distance, dtype: int64, 0        1132
 841         7
 1,039       7
 614         7
 917         7
 674         7
 744         6
 620         6
 682         6
 1,145    

In [27]:
# price variable have many values that need to be normalized before converting from cents to MXN
data.price.value_counts().head(20)

0         1083
4         1064
4,5        103
10          14
4,404        5
4,225        5
4,547        4
4,573        4
4,217        4
4,424        4
4,237        4
4,161        3
4,43         3
14,136       3
5,828        3
4,204        3
4,182        3
4,884        3
4,195        3
4,54         3
Name: price, dtype: int64

In [28]:
cancels = data.loc[((data['end_state'] == 'rider cancel')),price_context]

In [29]:
cancels['price'].value_counts()

0        1078
4          57
10          5
4,5         4
4,279       1
5,285       1
Name: price, dtype: int64

In [30]:
no_shows = data.loc[((data['end_state'] == 'no show')),price_context]

In [31]:
no_shows['price'].value_counts()

4         6
8,484     1
4,656     1
6,93      1
0         1
10,177    1
4,5       1
Name: price, dtype: int64

In [32]:
'''price_var = ['price_distance','price_duration','price_supplements']
data.loc[((data['price'] != '0') & ((len(data['price']) < max([len(data[v]) for v in price_var])))),price_context]'''

"price_var = ['price_distance','price_duration','price_supplements']\ndata.loc[((data['price'] != '0') & ((len(data['price']) < max([len(data[v]) for v in price_var])))),price_context]"

Before converting currency format (from cents to MXN), we need to analyze and normalize data patterns.
First of all, some Google investigation:

https://www.numbeo.com/taxi-fare/in/Mexico-City
https://www.taxi-calculator.com/taxi-rate-mexico_city/350
https://vanguardia.com.mx/articulo/uber-vs-cabify-vs-easy-taxi-cual-es-mejor
https://www.elfinanciero.com.mx/tech/que-te-conviene-taxi-o-uber

price_mini = (EASY) TAXI 8 MXN (800 cents)(basic fee + kms) - UBER 35 MXN (3500 cents) - CABIFY 40 MXN (4000 cents)
Additional fee for distance: 1 MXN (250m) - 4 MXN (1km)
Additional fee for waiting: = 100 MXN - 1h

1 MXN = 100 cents
1.000 cents = 0,47 EUR | 10.000 cents = 4.7 EUR | 100.000 cents = 47 EUR | 1.000.000 cents = 470 EUR
   10 MXN = 0,47 EUR   |    100 MXN = 4.7 EUR   |   1.000 MXN = 47 EUR   |    10.000 MXN = 470 EUR

4 = 4000 cents or 40 MXN --> concat '000'
10 = 1000(0) cents or 10(0) MXN --> concat '00' if rider cancel | --> concat '000' else
100 = 10000(0) cents or 100(0) MXN --> concat '00' if rider cancel | --> concat '000' else

4,5 = 4500 cents or 45 MXN --> concat '00'  & remove ','
16,2 = 1620(0) cents or 16(0).20 MXN --> (concat '0' if rider cancel | --> concat '00' else) & remove ',' 
160,2 = 16020(0) cents or 160(0).20 MXN --> (concat '0' if rider cancel | --> concat '00' else) & remove ','

4,20 = 4200(0) cents or 42(0) MXN  --> concat '0' else) & remove ','  
16,20 = 1620(0) cents or 16(0).20 MXN --> ( NO concat if rider cancel | --> concat '0' else) & remove ',' 
160,20 = 16020(0) cents or 160(0).20 MXN --> ( NO concat if rider cancel | --> concat '0' else) & remove ',' 

4,205 = 4205 cents or 42.05 MXN --> remove ','
14,200 = 14200 cents or 142 MXN --> remove ','
140,200 = 140200 cents or 1402 MXN --> remove ','

In [33]:
def clean_cents(s):
    rc = remove_comma(s)
    s = re.sub('^\d+\,\d{3}$',rc,s)
    return s

def clean_minimums(s):
    rc = remove_comma(s)
    s = re.sub('^\d{1}$',(s+'000'),s)
    s = re.sub('^\d{1}\,\d{1}$',(rc+'00'),s)
    s = re.sub('^\d{1}\,\d{2}$',(rc+'0'),s)
    return s

def clean_dropoffs(s):
    rc = remove_comma(s)
    s = re.sub('^\d{2,3}$',(s+'000'),s)
    s = re.sub('^\d{2,3}\,\d{1}$',(rc+'00'),s)
    s = re.sub('^\d{2,3}\,\d{2}$',(rc+'0'),s)
    return s

def clean_no_dropoffs(s):
    rc = remove_comma(s)
    s = re.sub('^\d{2,3}$',(s+'00'),s)
    s = re.sub('^\d{2,3}\,\d{1}$',(rc+'0'),s)
    s = re.sub('^\d{2,3}\,\d{2}$',rc,s) 
    return s

In [34]:
def clean_shit_string(df,var):
    for v in var:
        df[v] = df[v].astype('str')
        df[v] = df[v].apply(clean_cents)
        df[v] = df[v].apply(clean_minimums)
        df[v] = df[v].loc[(df['end_state'].isin(['rider cancel','no show']))] = df[v].apply(clean_no_dropoffs)
        df[v] = df[v].loc[(df['end_state']=='drop off')] = df[v].apply(clean_dropoffs)
    return df[var].head(50)

def clean_shit_MXN(df,var):
    for v in var:
        df[v] = df[v].apply(pd.to_numeric, errors='coerce')
        df[v] = df[v].apply(cents_to_MXN)
    return df[var].head(50)

In [35]:
price_var_to_clean = ['price','price_duration','price_distance','price_supplements']
clean_shit_string(data,price_var_to_clean)

/home/cecil/.local/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,price,price_duration,price_distance,price_supplements
0,5866,0000,5204,66200
1,0000,0000,0000,0000
2,4000,0000,3552,0000
3,0000,0000,0000,0000
4,4000,1200,2974,0000
5,4000,29000,3346,0000
6,0000,nan,nan,0000
7,6449,2082,4367,0000
8,0000,nan,nan,0000
9,8063,0000,4063,0000


In [36]:
clean_shit_MXN(data,price_var_to_clean)

,price,price_duration,price_distance,price_supplements
0,58.66,0.00,52.04,662.0
1,0.00,0.00,0.00,0.0
2,40.00,0.00,35.52,0.0
3,0.00,0.00,0.00,0.0
4,40.00,12.00,29.74,0.0
5,40.00,290.00,33.46,0.0
6,0.00,NaN,NaN,0.0
7,64.49,20.82,43.67,0.0
8,0.00,NaN,NaN,0.0
9,80.63,0.00,40.63,0.0


In [37]:
total_prices = data.loc[(data['price'] > 0)]
print(total_prices[price_var_to_clean].describe())
print('\n', total_prices['price'].value_counts().head(20))

             price  price_duration  price_distance  price_supplements
count  2761.000000     2729.000000     2729.000000        2761.000000
mean     68.426204       62.749689       72.430022          10.137917
std      59.777768      176.989486      113.126254          88.238424
min      10.000000        0.000000        0.000000           0.000000
25%      40.000000        0.000000       31.150000           0.000000
50%      43.900000        0.000000       41.690000           0.000000
75%      69.920000       12.350000       63.640000           0.000000
max    1745.540000      991.000000     1241.210000         922.000000

 40.00    1064
45.00     103
10.00      14
42.25       5
44.04       5
44.24       4
45.47       4
45.73       4
42.17       4
42.37       4
45.40       3
40.05       3
46.05       3
41.95       3
41.61       3
51.53       3
40.99       3
41.82       3
45.38       3
45.66       3
Name: price, dtype: int64


In [38]:
no_dropoffs_prices = data.loc[((data['price'] > 0)&(data['end_state'].isin(['rider cancel','no show'])))]
print(no_dropoffs_prices[price_var_to_clean].describe())
print('\n', no_dropoffs_prices['price'].value_counts().head(20))

            price  price_duration  price_distance  price_supplements
count   79.000000       47.000000            47.0               79.0
mean    40.419114       47.527234             0.0                0.0
std     12.148500      121.829303             0.0                0.0
min     10.000000        0.000000             0.0                0.0
25%     40.000000        0.000000             0.0                0.0
50%     40.000000        0.000000             0.0                0.0
75%     40.000000       15.650000             0.0                0.0
max    101.770000      635.000000             0.0                0.0

 40.00     63
45.00      5
10.00      5
101.77     1
52.85      1
69.30      1
46.56      1
84.84      1
42.79      1
Name: price, dtype: int64


In [39]:
dropoffs_prices = data.loc[((data['price'] > 0)&(data['end_state']=='drop off'))]
print(dropoffs_prices[price_var_to_clean].describe())
print('\n', dropoffs_prices['price'].value_counts().head(20))

             price  price_duration  price_distance  price_supplements
count  2682.000000     2682.000000     2682.000000        2682.000000
mean     69.251171       63.016450       73.699303          10.436536
std      60.420103      177.807871      113.702771          89.511616
min      10.000000        0.000000        0.000000           0.000000
25%      40.000000        0.000000       31.922500           0.000000
50%      44.260000        0.000000       42.020000           0.000000
75%      72.170000       12.237500       64.302500           0.000000
max    1745.540000      991.000000     1241.210000         922.000000

 40.00    1001
45.00      98
10.00       9
42.25       5
44.04       5
44.24       4
45.47       4
45.73       4
42.17       4
42.37       4
40.99       3
40.05       3
46.05       3
41.95       3
45.40       3
51.53       3
41.82       3
45.38       3
45.66       3
50.75       3
Name: price, dtype: int64


In [40]:
seconds_var = ['duration','driver_waiting_time','rider_waiting_time']
convert_col_to_numeric(data,seconds_var)
second_to_min(data,seconds_var)

,start_lat,start_lon,end_lat,end_lon,price,price_distance,price_duration,duration,rider_waiting_time,driver_waiting_time,price_supplements,rating,duration_min,driver_waiting_time_min,rider_waiting_time_min
count,3864.000000,3864.000000,3807.000000,3807.000000,3844.000000,2812.000000,2812.000000,2514.000000,2405.000000,2612.000000,3864.000000,3864.000000,2514.000000,2612.000000,2405.000000
mean,19.412159,-99.195443,19.410835,-99.197221,49.147958,70.682575,60.921248,352.604216,398.552599,139.151608,7.243993,8.289596,5.876737,2.319193,6.642543
std,0.028146,0.040832,0.030001,0.044896,59.279811,113.349455,174.674982,353.279282,223.464662,190.048695,74.725170,1.265744,5.887988,3.167478,3.724411
min,19.290513,-99.288274,19.196463,-99.655665,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.050000
25%,19.405320,-99.207814,19.379486,-99.253980,0.000000,29.667500,0.000000,0.000000,227.000000,11.000000,0.000000,8.000000,0.000000,0.183333,3.783333
50%,19.422749,-99.175834,19.422749,-99.178435,40.000000,41.285000,0.000000,350.000000,356.000000,52.000000,0.000000,9.000000,5.833333,0.866667,5.933333
75%,19.426613,-99.170483,19.429759,-99.171955,50.862500,62.377500,11.267500,679.000000,534.000000,192.000000,0.000000,9.000000,11.316667,3.200000,8.900000
max,19.513544,-99.051114,19.517451,-99.048817,1745.540000,1241.210000,991.000000,999.000000,999.000000,999.000000,922.000000,10.000000,16.650000,16.650000,16.650000


In [41]:
split_datetime(data, 'start_at')

,start_at,start_at_year,start_at_month,start_at_date,start_at_time
0,2017-12-16 11:08:59,2017,12,2017-12-16,11:08:59
1,2017-12-16 11:06:16,2017,12,2017-12-16,11:06:16
2,2017-11-24 22:58:37,2017,11,2017-11-24,22:58:37
3,2017-12-16 09:17:08,2017,12,2017-12-16,09:17:08
4,2017-11-24 18:11:00,2017,11,2017-11-24,18:11:00
5,2017-11-24 15:34:53,2017,11,2017-11-24,15:34:53
6,2017-11-24 15:10:23,2017,11,2017-11-24,15:10:23
7,2017-11-24 15:09:29,2017,11,2017-11-24,15:09:29
8,2017-11-24 14:31:58,2017,11,2017-11-24,14:31:58
9,2017-11-24 14:20:00,2017,11,2017-11-24,14:20:00


In [42]:
split_datetime(data, 'end_at')

,end_at,end_at_year,end_at_month,end_at_date,end_at_time
0,2017-12-16 11:37:32,2017,12,2017-12-16,11:37:32
1,2017-12-16 11:07:21,2017,12,2017-12-16,11:07:21
2,2017-11-24 23:27:05,2017,11,2017-11-24,23:27:05
3,2017-12-16 09:17:30,2017,12,2017-12-16,09:17:30
4,2017-11-24 18:31:31,2017,11,2017-11-24,18:31:31
5,2017-11-24 15:53:30,2017,11,2017-11-24,15:53:30
6,2017-11-24 15:10:42,2017,11,2017-11-24,15:10:42
7,2017-11-24 15:47:42,2017,11,2017-11-24,15:47:42
8,2017-11-24 14:32:56,2017,11,2017-11-24,14:32:56
9,2017-11-24 14:36:44,2017,11,2017-11-24,14:36:44


In [ ]:
data['calc_duration'] = data['end_at'] -  data['start_at']

In [67]:
def calc_dist_km(d):
    return distance.distance(d).km

def calc_dist_miles(d):
    return distance.distance(d).miles

def calculate_distances(df,g1,g2):
    for c1,c2 in g1:
        d=tuple(df[g1],df[g2])
        df['calc_distance_km'] = d.apply(calc_dist_km)
        df['calc_distance_miles'] = d.apply(calc_dist_miles)

In [68]:
data['geopoint_1'] = ('start_lat','start_lon')
data['geopoint_2'] = ('end_lat', 'end_lon')
data['calc_distance_km'] = distance.distance(data['geopoint_1'],data['geopoint_2']).km
data['calc_distance_miles'] = distance.distance(data['geopoint_1'],data['geopoint_2']).miles

ValueError: Length of values does not match length of index

In [ ]:
check_distance = ['start_lat','start_lon','coords_1','end_lat', 'end_lon','coords_2','distance','calc_distance_km','calc_distance_miles']
data[check_distance].head(10)

In [43]:
export_clean_csv(data,'clean_data')

### Data wrangling: Decisions Summary:

* **Missing values:** 
- replace with '0': distance, driver_waiting_time, rider_waiting_time, price, price_duration, price_distance, price supplements
- keep 'NaN': end_lat & end_long (--> can be canceled or rider not found)

* **Variables types:** 
- object: none needed
- category: all _type & _state variables & source
- numeric: 
    + all prices+rating (float) variables
    + duration/time (float or int) variables
    + all geolocation variables: _lon & _lat variables (start & end)
    + datetime : _at variables (start & end)

* **Convert measures /Add new variables:** 
- datetime : extract year, month, day, time
- duration & time var --> second to minutes format
- price var --> 1) normalize depending on end_state and then convert cents to MXN

* **Change label:** 
- vehicle_type_id: 2 types --> rename with 'A'/'B'
- driver_waiting_time & rider_waiting_time: ' --> from categorical to time format

* **Drop variables:** 
- few values: discount
- not relevant: unnamed_1

Here I just tried to re-factorize my coding for prices data wrangling  following the "kata" spirit, by splitting 
the whole taff in small tasks - but for the moment it's pending to close because I spend many hour with the other way and do not have enought time considering that Tableau charts, dashboards and stories design is still pending.

In [44]:
# concatenate a specific number of zeros to a string (after) 
def concat0(s, n_zeros):
    zeros = n_zeros * '0'
    return s + zeros
    
s001 = concat0('4,5', 2)
print (s001, type(s001))

4,500 <class 'str'>


In [45]:
# check if string has a comma
def check_comma(s):
    if ',' in s:
        return True
    else:
        return False

In [46]:
# return number of digits after the comma (=decimals)
def count_decimals(s):
    if check_comma(s):
        lista = s.split(',')
        return len(lista[1])
    else:
        return 0
print(count_decimals('4,1'))

1


In [47]:
# return number of digits before the comma (=integers)
def count_integers(s):
    i=s[:s.index(',')]
    return len(i)

f = count_integers('544,4')
print (f, type(f))

3 <class 'int'>


In [48]:
# normalize values of prices from string
def normalize_dropoff_price(s):
    dec = count_decimals(s)
    zeros = 3-dec
    if dec != 0: # if decimals remove comma
        s = s.replace(',','') 
    return concat0(s, zeros)

print (normalize_dropoff_price('4,112'))

4112


In [49]:
# clean all this price data jungle
def clean_shit(df,lab,var):
    if df.loc[(df[lab].isin(['rider cancel','no show'])),[lab]]:
        if count_decimals()
    
    elif df.loc[(df[lab].isin(['drop off'])),[lab]]:
    
    else:
        df[lab]
df.loc[(df[lab] == 'driver cancel'),[lab]]   
    
check_end_state(data,'end_state')

SyntaxError: invalid syntax (<ipython-input-49-aa2bc2e731f2>, line 4)